In [1]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1748278147702_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Funciones PySpark
from pyspark.sql.functions import regexp_replace, col, udf, explode, split, lower, size, trim, col, arrays_zip
from pyspark.sql.types import StringType, ArrayType,MapType,IntegerType,DoubleType,StructType, StructField,DateType

# Componentes de Spark NLP
from sparknlp.base import DocumentAssembler, Pipeline, Finisher
from sparknlp.annotator import Tokenizer, PerceptronModel, LemmatizerModel, RegexMatcher,StopWordsCleaner

from pyspark.ml import Pipeline
from pyspark.sql.functions import udf,upper,concat_ws,col, count
from sparknlp.common import ExternalResource
from pyspark.sql.functions import col, count
from pyspark.ml.feature import StringIndexer, NGram, CountVectorizer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml.stat import ChiSquareTest


from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.stat import Correlation
import numpy as np

import re

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
bucket    = "proyectointegrador1017"
raw       = f"s3://{bucket}/raw"
paths = {
    "base_prediccion":     f"{raw}/Base_Prediccion_28092021.csv",
    "variables_sel":       f"{raw}/Variables_seleccionadas.csv",
    "diccionario_inicial": f"{raw}/DiccionarioInicial.csv"
}

# Carga cada CSV en un DataFrame Spark
df_base = spark.read.option("header", "true") \
                .option("sep", ",") \
                .option("quote", '"') \
                .option("multiLine", "true") \
                .option("escape", "\\") \
                .option("inferSchema", "true") \
                .csv(paths["base_prediccion"])

df_vars = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv(paths["variables_sel"])

df_dicc = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv(paths["diccionario_inicial"])


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
bucket    = "proyectointegrador1017"
raw       = f"s3://{bucket}/raw"
path_base = f"{raw}/Base_Prediccion_28092021.csv"

# Esquema explícito
schema = StructType([
    StructField("ID_ORDEM_SERVICO", StringType(), True),
    StructField("ID_UC", StringType(), True),
    StructField("Estado Instalacion", StringType(), True),
    StructField("Uso", StringType(), True),
    StructField("Actividad Economica", StringType(), True),
    StructField("Fecha Generacion", DateType(), True),
    StructField("Fecha Ejecucion", DateType(), True),
    StructField("FechaDescargue", DateType(), True),
    StructField("Fecha Ultima Lectura", DateType(), True),
    StructField("Estado OT", StringType(), True),
    StructField("Observacion", StringType(), True),
    StructField("Efectividad", StringType(), True),
    StructField("Causa Efectividad", StringType(), True),
    StructField("Causa Evento", StringType(), True),
    StructField("Consumo promedio", DoubleType(), True),
    StructField("Lectura en Visita", DoubleType(), True),
    StructField("Ultimo Valor Leido", DoubleType(), True),
    StructField("Oportunidad Ejecucion", DoubleType(), True),
    StructField("Oportunidad Descargue", DoubleType(), True),
    StructField("Diferencia Lectura", DoubleType(), True),
    StructField("Dias entre lecturas", DoubleType(), True),
    StructField("Promedio Consumo dia", DoubleType(), True),
    StructField("Analisis Lectura", StringType(), True),
    StructField("Promedio Dia Historico", DoubleType(), True),
    StructField("Menor a Promedio", StringType(), True),
    StructField("Score Lectura", DoubleType(), True),
    StructField("Score Promedio", DoubleType(), True),
    StructField("Total Puntaje", DoubleType(), True),
    StructField("ID_Accion", StringType(), True),
    StructField("Nombre Accion", StringType(), True)
])

#  Lectura usando el esquema
df_datos = spark.read.option("header", "true") \
                .option("sep", ",") \
                .option("quote", '"') \
                .option("multiLine", "true") \
                .option("escape", "\\") \
                .option("inferSchema", "true") \
                .schema(schema) \
                .csv(path_base)

#  Verifica tipos
df_datos.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ID_ORDEM_SERVICO: string (nullable = true)
 |-- ID_UC: string (nullable = true)
 |-- Estado Instalacion: string (nullable = true)
 |-- Uso: string (nullable = true)
 |-- Actividad Economica: string (nullable = true)
 |-- Fecha Generacion: date (nullable = true)
 |-- Fecha Ejecucion: date (nullable = true)
 |-- FechaDescargue: date (nullable = true)
 |-- Fecha Ultima Lectura: date (nullable = true)
 |-- Estado OT: string (nullable = true)
 |-- Observacion: string (nullable = true)
 |-- Efectividad: string (nullable = true)
 |-- Causa Efectividad: string (nullable = true)
 |-- Causa Evento: string (nullable = true)
 |-- Consumo promedio: double (nullable = true)
 |-- Lectura en Visita: double (nullable = true)
 |-- Ultimo Valor Leido: double (nullable = true)
 |-- Oportunidad Ejecucion: double (nullable = true)
 |-- Oportunidad Descargue: double (nullable = true)
 |-- Diferencia Lectura: double (nullable = true)
 |-- Dias entre lecturas: double (nullable = true)
 |-- Promedio C

In [48]:
df_datos.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------+--------------------+-----------+--------------------+----------------+---------------+--------------+--------------------+-----------------+--------------------+-----------+--------------------+--------------------+----------------+-----------------+------------------+---------------------+---------------------+------------------+-------------------+--------------------+----------------+----------------------+--------------------+-------------+--------------+-------------+---------+--------------------+
|ID_ORDEM_SERVICO|   ID_UC|  Estado Instalacion|        Uso| Actividad Economica|Fecha Generacion|Fecha Ejecucion|FechaDescargue|Fecha Ultima Lectura|        Estado OT|         Observacion|Efectividad|   Causa Efectividad|        Causa Evento|Consumo promedio|Lectura en Visita|Ultimo Valor Leido|Oportunidad Ejecucion|Oportunidad Descargue|Diferencia Lectura|Dias entre lecturas|Promedio Consumo dia|Analisis Lectura|Promedio Dia Historico|    Menor a Promedio|S

In [49]:
df_diccionario = df_dicc

# Carga del CSV de stop-words como DataFrame
path_stopwords = "s3://proyectointegrador1017/raw/Stopwords.csv"
df_Stop_words = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .option("delimiter", ";") \
    .csv(path_stopwords)

# Verificar contenido
df_Stop_words.printSchema()
df_Stop=df_Stop_words.select('Stopword')
df_Stop.show(5, truncate=False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Stopword: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)

+--------+
|Stopword|
+--------+
|1       |
|3       |
|?1      |
|aal     |
|abb     |
+--------+
only showing top 5 rows

In [50]:
def depurar_Observacion(observacion):
    try:
        if not isinstance(observacion, str):
            return None

        inicio = observacion.find("FECHA VIS")
        if inicio == -1:
            inicio = observacion.find("FECHA VISITA    PAP")
            if inicio == -1:
                return None
            else:
                inicio += len("FECHA VISITA    PAP")
        else:
            inicio += len("FECHA VIS")

        subcadena = observacion[inicio:]

        fin = subcadena.find("RAZON")
        if fin == -1:
            return None

        resultado = subcadena[:fin].strip()
        return resultado
    except Exception:
        return None


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:


def limpiar_caracteres(cadena):
    if not isinstance(cadena, str):
        return ""
    patron = r'[^A-Za-z0-9]+'
    return re.sub(patron, ' ', cadena).strip()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
def quitar_informacion_sello(cadena):
    if not isinstance(cadena, str):
        return ""
    patron = r'#\w+\d+'
    return re.sub(patron, '', cadena).strip()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
def quitar_numericos(cadena):
    if not isinstance(cadena, str):
        return ""
    patron = r'[0-9]+'
    return re.sub(patron, '', cadena).strip()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
def depurar_marca_causa_evento(observacion):
    if not isinstance(observacion, str):
        return ""
    posicion = observacion.find("*CausaEvento")
    if posicion == -1:
        return observacion.strip()
    return observacion[:posicion].strip()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
def depurar_usuario(observacion):
    if not isinstance(observacion, str):
        return ""
    observacion_mayus = observacion.upper()
    posicion = observacion_mayus.find("USUARIO: EPM ")
    if posicion == -1:
        posicion = observacion_mayus.find("USUARIO EPM")
        if posicion == -1:
            return observacion.strip()
        return observacion.strip()
    return observacion[:posicion].strip()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
# Registrar UDFs
limpiar_caracteres_udf = udf(limpiar_caracteres, StringType())
quitar_informacion_sello_udf = udf(quitar_informacion_sello, StringType())
quitar_numericos_udf = udf(quitar_numericos, StringType())
depurar_marca_causa_evento_udf = udf(depurar_marca_causa_evento, StringType())
depurar_usuario_udf = udf(depurar_usuario, StringType())

# Aplicar UDFs en cadena
df_limpio = df_datos \
    .withColumn("Observacion_limpia_1", limpiar_caracteres_udf(col("Observacion"))) \
    .withColumn("Observacion_limpia_2", quitar_informacion_sello_udf(col("Observacion_limpia_1"))) \
    .withColumn("Observacion_limpia_3", quitar_numericos_udf(col("Observacion_limpia_2"))) \
    .withColumn("Observacion_limpia_4", depurar_marca_causa_evento_udf(col("Observacion_limpia_3"))) \
    .withColumn("Observacion_final", depurar_usuario_udf(col("Observacion_limpia_4")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
# Mostrar resultado
df_limpio.select('Observacion','observacion_final').show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [58]:
stopwords_list = [row["Stopword"] for row in df_Stop.collect()]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
stopwords_set = set(stopwords_list)

def quitar_stopwords(texto):
    if texto:
        palabras = texto.split()
        filtradas = [p for p in palabras if p.lower() not in stopwords_set]
        return " ".join(filtradas)
    return ""

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
# Registrar la función como UDF
quitar_stopwords_udf = udf(quitar_stopwords, StringType())

# Aplicarla sobre la columna 'observacion'
df_limpio = df_limpio.withColumn("observacion_sin_stopwords", quitar_stopwords_udf(col("observacion")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##Aqui va nlp cuando lo solucionemos

In [62]:
dict_sinonimos = dict(df_diccionario.select("Palabra", "Sinonimo").rdd.map(lambda row: (row["Palabra"], row["Sinonimo"])).collect())


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
def reemplazar_sinonimos(texto):
    if not isinstance(texto, str):
        return []
    palabras = texto.split()
    resultado = [dict_sinonimos.get(p, p) for p in palabras]
    return resultado


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
df_limpio.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ID_ORDEM_SERVICO', 'ID_UC', 'Estado Instalacion', 'Uso', 'Actividad Economica', 'Fecha Generacion', 'Fecha Ejecucion', 'FechaDescargue', 'Fecha Ultima Lectura', 'Estado OT', 'Observacion', 'Efectividad', 'Causa Efectividad', 'Causa Evento', 'Consumo promedio', 'Lectura en Visita', 'Ultimo Valor Leido', 'Oportunidad Ejecucion', 'Oportunidad Descargue', 'Diferencia Lectura', 'Dias entre lecturas', 'Promedio Consumo dia', 'Analisis Lectura', 'Promedio Dia Historico', 'Menor a Promedio', 'Score Lectura', 'Score Promedio', 'Total Puntaje', 'ID_Accion', 'Nombre Accion', 'Observacion_limpia_1', 'Observacion_limpia_2', 'Observacion_limpia_3', 'Observacion_limpia_4', 'Observacion_final', 'observacion_sin_stopwords']

In [65]:
reemplazo_udf = udf(reemplazar_sinonimos, ArrayType(StringType()))

df_final = df_limpio.withColumn("Observacion_Ok", reemplazo_udf(col("observacion_sin_stopwords")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
#vamos a agrupar las causas eventos en 3 categorias: No revisado, No fraude o Anomalia
def agrupar_causaevento(causa):
    if (causa== 'No Aplica' or causa=='No autorizan trabajo' or causa=='Otras Fuentes' or causa=='Información errada'):
        evento='No revisado'
    elif (causa=='No fraude'):
        evento='Normal'
    else:
        evento='Anomalia'
    return evento

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
agrupar_udf = udf(agrupar_causaevento, StringType())
final_df= df_final.withColumn("evento", agrupar_udf(df_final["Causa Evento"]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
final_df.select("Causa Evento", "evento").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+
|        Causa Evento|     evento|
+--------------------+-----------+
|Anomalia - Instal...|   Anomalia|
|           No fraude|     Normal|
|Anomalia - Medido...|   Anomalia|
|           No Aplica|No revisado|
|           No fraude|     Normal|
|           No fraude|     Normal|
|           No fraude|     Normal|
|           No fraude|     Normal|
|           No fraude|     Normal|
|           No fraude|     Normal|
|           No fraude|     Normal|
|           No fraude|     Normal|
|           No Aplica|No revisado|
|Anomalia - Medido...|   Anomalia|
|           No fraude|     Normal|
|           No fraude|     Normal|
|           No fraude|     Normal|
|Anomalia - Instal...|   Anomalia|
|           No fraude|     Normal|
|           No fraude|     Normal|
+--------------------+-----------+
only showing top 20 rows

In [71]:
def eliminar_pares_correlacionados(df_spark, columnas_categoricas, umbral=0.05):
    from pyspark.ml.linalg import Vectors
    from pyspark.ml.feature import VectorAssembler
    from pyspark.sql import DataFrame

    df = df_spark.select(columnas_categoricas).dropna()

    # Convertir columnas categóricas a índices numéricos
    for colname in columnas_categoricas:
        indexer = StringIndexer(inputCol=colname, outputCol=colname + "_idx").setHandleInvalid("skip")
        df = indexer.fit(df).transform(df)

    idx_cols = [col + "_idx" for col in columnas_categoricas]

    # Contamos pares dependientes y eliminamos uno de cada par
    dependientes = set()
    columnas_validas = idx_cols.copy()

    for i in range(len(idx_cols) - 1):
        for j in range(i + 1, len(idx_cols)):
            assembler = VectorAssembler(inputCols=[idx_cols[j]], outputCol="features")
            chi_input = assembler.transform(df.select(idx_cols[i], idx_cols[j]).withColumnRenamed(idx_cols[i], "label"))
            chi_result = ChiSquareTest.test(chi_input, "features", "label").head()
            p_value = chi_result.pValues[0]

            if p_value < umbral:
                # Hay dependencia significativa: eliminar la de menor cardinalidad
                col1 = idx_cols[i]
                col2 = idx_cols[j]
                card1 = df.select(col1).distinct().count()
                card2 = df.select(col2).distinct().count()

                if card1 >= card2:
                    dependientes.add(col2)
                else:
                    dependientes.add(col1)

    columnas_finales = [col.replace("_idx", "") for col in columnas_validas if col not in dependientes]
    return columnas_finales


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
columnas = ['Uso', 'Actividad Economica','Analisis Lectura', 'Menor a Promedio']
columnas_Categorica_resultado = eliminar_pares_correlacionados(df_final, columnas)
print(columnas_Categorica_resultado)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Actividad Economica']

In [73]:
def convertir_columna_a_numerico(df, colname, nueva_colname=None):
    if not nueva_colname:
        nueva_colname = colname
    return (df.withColumn(nueva_colname, regexp_replace(col(colname), "[$,%]", ""))
              .withColumn(nueva_colname, regexp_replace(col(nueva_colname), ",", ""))
              .withColumn(nueva_colname, col(nueva_colname).cast("double")))

def convertir_varias_columnas_a_numerico(df, columnas):
    """
    Limpia y convierte varias columnas a tipo numérico en PySpark.
    """
    for colname in columnas:
        df = convertir_columna_a_numerico(df, colname)
    return df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
columnas_numericas = ['Consumo promedio', 'Lectura en Visita', 'Ultimo Valor Leido', 'Diferencia Lectura','Promedio Consumo dia','Score Lectura', 'Score Promedio', 'Total Puntaje']
df_numericas = convertir_varias_columnas_a_numerico(final_df, columnas_numericas)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
def eliminar_correlacionadas_spark(df_spark, columnas, umbral=0.7):
    """
    Elimina variables numéricas altamente correlacionadas (> umbral)
    """
    # 1. Reemplazar nulos (puedes hacer un dropna o imputación)
    df = df_spark.select(columnas).dropna()

    # 2. Ensamblar vector
    assembler = VectorAssembler(inputCols=columnas, outputCol="features_raw")
    df_vector = assembler.transform(df)

    # 3. Escalar los datos
    scaler = StandardScaler(inputCol="features_raw", outputCol="features", withStd=True, withMean=True)
    scaler_model = scaler.fit(df_vector)
    df_scaled = scaler_model.transform(df_vector)

    # 4. Calcular matriz de correlación
    corr_matrix = Correlation.corr(df_scaled, "features", method="pearson").head()[0].toArray()

    # 5. Eliminar una variable de cada par altamente correlacionado
    keep = set(columnas)
    eliminar = set()

    for i in range(len(columnas)):
      for j in range(i+1, len(columnas)):
          if abs(corr_matrix[i, j]) > umbral:
              var_i = df.select(columnas[i]).agg({columnas[i]: "variance"}).first()[0]
              var_j = df.select(columnas[j]).agg({columnas[j]: "variance"}).first()[0]
              if var_i >= var_j:
                  eliminar.add(columnas[j])
              else:
                  eliminar.add(columnas[i])

    columnas_final = [col for col in columnas if col not in eliminar]
    return columnas_final


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
columnas_numericas = ['Consumo promedio', 'Lectura en Visita', 'Ultimo Valor Leido', 'Diferencia Lectura','Promedio Consumo dia','Score Lectura', 'Score Promedio', 'Total Puntaje']
columnas_seleccionadas = eliminar_correlacionadas_spark(df_numericas, columnas_numericas)
print(columnas_seleccionadas)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Consumo promedio', 'Ultimo Valor Leido', 'Diferencia Lectura', 'Total Puntaje']

In [77]:
cols_seleccionadas = ['ID_UC']+columnas_Categorica_resultado + columnas_seleccionadas + ['Observacion_Ok', 'evento']

# Seleccionar columnas del DataFrame
df_entrenamiento = df_numericas.select(*cols_seleccionadas)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
def categorizar_y(x):
    try:
        if x == 'Normal':
            return 0
        elif x == 'Anomalia':
            return 1
        else:
            return 2
    except:
        return 2

# Regístrala como UDF
categorizar_y_udf = udf(categorizar_y, IntegerType())

# Aplica al DataFrame (por ejemplo sobre la columna 'evento')
df_entrenamiento = df_entrenamiento.withColumn("evento_cat", categorizar_y_udf("evento"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
df_entrenamiento.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+----------------+------------------+------------------+-------------+--------------------+-----------+----------+
|   ID_UC| Actividad Economica|Consumo promedio|Ultimo Valor Leido|Diferencia Lectura|Total Puntaje|      Observacion_Ok|     evento|evento_cat|
+--------+--------------------+----------------+------------------+------------------+-------------+--------------------+-----------+----------+
|12285549|Residencial estra...|             2.0|               0.0|               0.0|         17.0|[Se, encuentra, v...|   Anomalia|         1|
|12285712|Expendio de bebid...|            12.0|            3172.0|               8.0|          4.0|[Jaramillo, Medid...|     Normal|         0|
|12285932|Residencial estra...|             6.0|               3.0|              14.0|          4.0|[Yonatan, Giraldo...|   Anomalia|         1|
|12286097|Otras actividades...|            12.0|           50739.0|           50739.0|          4.0|[SEBASTIAN, BUILE...|No revisa

In [80]:
ngram = NGram(n=2, inputCol="Observacion_Ok", outputCol="bigrams")

# 4. Vectorizar bigramas
vectorizer = CountVectorizer(inputCol="bigrams", outputCol="features")

# Crear Pipeline
pipeline = Pipeline(stages=[ngram, vectorizer])

# Ajustar y transformar
modelo = pipeline.fit(df_entrenamiento)
df_transformado = modelo.transform(df_entrenamiento)

# Mostrar resultado
df_transformado.select("Observacion_Ok", "bigrams", "features").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [81]:
from pyspark.sql.functions import concat_ws, col

# 1) Aplanar arrays
array_cols = ["Observacion_Ok", "bigrams"]
df_flat = df_transformado
for c in array_cols:
    df_flat = df_flat.withColumn(
        f"{c}_str",
        concat_ws(" ", col(c))
    ).drop(c)

# 2) Quitar la columna 'features' de tipo Struct/vector
df_flat = df_flat.drop("features")

# 3) Definir ruta en trusted
path_trusted_csv = "s3://proyectointegrador1017/trusted/Base_Entrenamiento_CSV"

# 4) Guardar en CSV
df_flat \
    .write \
    .mode("overwrite") \
    .option("header", True) \
    .csv(path_trusted_csv)

print(f"df_transformado limpio y guardado en CSV en: {path_trusted_csv}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df_transformado limpio y guardado en CSV en: s3://proyectointegrador1017/trusted/Base_Entrenamiento_CSV

In [82]:
# 1. Columnas categóricas, numéricas, y features de texto
categoricas = ['Actividad Economica']
numericas = ['Consumo promedio', 'Ultimo Valor Leido','Diferencia Lectura','Total Puntaje']
texto_vector = 'features'  # salida CountVectorizer u otro transformador
label_col = 'evento_cat'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:

cols_seleccionadas2 = categoricas + numericas + ['features', 'evento_cat']

# Seleccionar columnas del DataFrame
df_selecciondas = df_transformado.select(*cols_seleccionadas2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
df_selecciondas = df_selecciondas.dropna(subset=cols_seleccionadas2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
# 1. Columnas categóricas, numéricas, y features de texto



# Convertir las columnas numéricas a DoubleType BEFORE splitting the data
for c in numericas:
    df_selecciondas = df_selecciondas.withColumn(c, col(c).cast(DoubleType()))

# 2. Indexar categóricas
indexers = [
    StringIndexer(inputCol=c, outputCol=c + "_idx", handleInvalid="keep")
    for c in categoricas
]

# 3. OneHotEncode
encoder = OneHotEncoder(
    inputCols=[c + "_idx" for c in categoricas],
    outputCols=[c + "_vec" for c in categoricas],
    handleInvalid="keep"
)

# 4. VectorAssembler para numéricas
assembler_numeric = VectorAssembler(
    inputCols=numericas,
    outputCol="numerical_assembled",
    handleInvalid="keep"
)

# 5. Escalar numéricas
scaler = StandardScaler(
    inputCol="numerical_assembled",
    outputCol="numerical_scaled"
)

# 6. Combinar numéricas escaladas, categóricas codificadas y texto
assembler_final = VectorAssembler(
    inputCols=["numerical_scaled"] + [c + "_vec" for c in categoricas] + [texto_vector],
    outputCol="all_features",
    handleInvalid="skip"
)

# 7. Regresión logística
lr = LogisticRegression(featuresCol="all_features", labelCol="evento_cat")

# 8. Pipeline completo
pipeline = Pipeline(stages=indexers + [encoder, assembler_numeric, scaler, assembler_final, lr])

# 9. Dividir base en train y test - Use the df_transformado after type conversion
train_df, test_df = df_selecciondas.randomSplit([0.8, 0.2], seed=42) # Changed from df_limpio

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [86]:
modelo = pipeline.fit(train_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
predicciones = modelo.transform(test_df)

# Mostrar algunas predicciones
predicciones.select(label_col, "prediction", "probability").show(5, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+---------------------------------------------------------------+
|evento_cat|prediction|probability                                                    |
+----------+----------+---------------------------------------------------------------+
|1         |1.0       |[5.114017182955336E-9,0.9999999470385147,4.784746809936241E-8] |
|0         |0.0       |[0.9999999850110954,5.80668889531058E-9,9.182215742108921E-9]  |
|0         |0.0       |[0.9999981850036765,1.2325078787829488E-6,5.824884446424269E-7]|
|0         |0.0       |[0.9999996884941256,1.596685824774958E-7,1.518372917976569E-7] |
|0         |0.0       |[0.9999993419153371,3.288859152378159E-7,3.2919874762477983E-7]|
+----------+----------+---------------------------------------------------------------+
only showing top 5 rows

In [88]:
# Accuracy
evaluator_accuracy = MulticlassClassificationEvaluator(
    labelCol="evento_cat", predictionCol="prediction", metricName="accuracy"
)

# F1 Score
evaluator_f1 = MulticlassClassificationEvaluator(
    labelCol="evento_cat", predictionCol="prediction", metricName="f1"
)

# Calcular métricas
accuracy = evaluator_accuracy.evaluate(predicciones)
f1 = evaluator_f1.evaluate(predicciones)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy: 0.8964
F1 Score: 0.8941

In [89]:
def des_categorizar_y(x):
    if x == 0:
        return 'Normal'
    elif x == 1:
        return 'Anomalia'
    else:
        return 'No revisado'

# Registrar como UDF
des_categorizar_y_udf = udf(des_categorizar_y, StringType())

# Aplicar al DataFrame
df_con_clase = predicciones.withColumn("clase", des_categorizar_y_udf("prediction"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
df_con_clase.select('Clase','evento_cat').show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+
|   Clase|evento_cat|
+--------+----------+
|Anomalia|         1|
|  Normal|         0|
|  Normal|         0|
|  Normal|         0|
|  Normal|         0|
+--------+----------+
only showing top 5 rows

In [91]:
df_con_clase.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Actividad Economica', 'Consumo promedio', 'Ultimo Valor Leido', 'Diferencia Lectura', 'Total Puntaje', 'features', 'evento_cat', 'Actividad Economica_idx', 'Actividad Economica_vec', 'numerical_assembled', 'numerical_scaled', 'all_features', 'rawPrediction', 'probability', 'prediction', 'clase']

In [94]:


# 3) Define la lista de columnas a eliminar (nombres limpios)
cols_to_drop = [
    "actividad_economica",
    "consumo_promedio",
    "ultimo_valor_leido",
    "diferencia_lectura",
    "total_puntaje",
    "prediction",
    "clase"
]

# 4) Elimina las columnas con unpacking
df_flat = df_flat.drop(*cols_to_drop)

# 5) Define la ruta en refined para tus CSV de predicciones
refined_path = "s3://proyectointegrador1017/refined/predicciones_csv/"

# 6) Guarda el DataFrame sin esas columnas en un solo CSV
df_flat.coalesce(1) \
       .write \
       .mode("overwrite") \
       .option("header", True) \
       .csv(refined_path)

print(f"✅ DataFrame limpio guardado en: {refined_path}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

? DataFrame limpio guardado en: s3://proyectointegrador1017/refined/predicciones_csv/